# Table of Contents
 <p><div class="lev1"><a href="#Summary"><span class="toc-item-num">1&nbsp;&nbsp;</span>Summary</a></div><div class="lev1"><a href="#Imports"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></div><div class="lev1"><a href="#Load-data"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load data</a></div><div class="lev1"><a href="#Add-SIFTS"><span class="toc-item-num">4&nbsp;&nbsp;</span>Add SIFTS</a></div><div class="lev1"><a href="#Add-Partner"><span class="toc-item-num">5&nbsp;&nbsp;</span>Add Partner</a></div><div class="lev1"><a href="#Save-to-database"><span class="toc-item-num">6&nbsp;&nbsp;</span>Save to database</a></div>

# Summary

This is an Alanine-scanning dataset!

# Imports

In [1]:
%run imports.ipynb

2016-07-17 21:12:08.596166


In [2]:
NOTEBOOK_NAME = 'kortemme_baker'
os.makedirs(NOTEBOOK_NAME, exist_ok=True)
os.environ['NOTEBOOK_NAME'] = NOTEBOOK_NAME

os.environ['DB_PORT'] = '8308'

In [3]:
%run mysqld.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2016-07-17 21:12:08.820091


In [4]:
# Start a database connection
db_remote = datapkg.MySQL(
    os.environ['DATAPKG_CONNECTION_STR'], 
    shared_folder=os.environ['NOTEBOOK_NAME'],
    storage_host=None,
    echo=False,
    db_engine='InnoDB'
)

# Load data

Load data and format info columns.

In [5]:
kortemme_baker_df = pd.read_csv(
    '../downloads/rosetta_ddg/input/csv/kortemme_baker.csv',
    na_values=['None', ''])

In [6]:
kortemme_baker_df.shape

(380, 13)

In [7]:
display(kortemme_baker_df.head())
print(kortemme_baker_df.shape)

,PDB_ID,Mutated_partner,Chain_ID,PDB_res#,Amino_acid,DDG_calc,DDG_obs,"Interface(O:no,1:yes)",PDBPosID,PubMedID,Location,FixedPDB_ID,FixedPDBResidueID
0,1A22,hGH,A,14,M,0.00,0.1,0,1A2214,7504735,Table_2,NaN,NaN
1,1A22,hGH,A,18,H,1.84,-0.5,1,1A2218,7504735,Table_2,NaN,NaN
2,1A22,hGH,A,21,H,0.78,0.2,1,1A2221,7504735,Table_2,NaN,NaN
3,1A22,hGH,A,22,Q,0.07,-0.2,1,1A2222,7504735,Table_2,NaN,NaN
4,1A22,hGH,A,25,F,1.30,-0.4,1,1A2225,7504735,Table_2,NaN,NaN


(380, 13)


In [8]:
kortemme_baker_df = kortemme_baker_df.rename(columns={
        'PDB_ID': 'pdb_id',
        'Chain_ID': 'pdb_chain',
        'Amino_acid': 'pdb_aa_wt',
        'PDB_res#': 'pdb_resnum',
    })

In [9]:
# Add required columns
kortemme_baker_df['pdb_id'] = kortemme_baker_df['pdb_id'].str.lower()
kortemme_baker_df['partner_pdb_chain'] = np.nan
kortemme_baker_df['pdb_aa_mut'] = 'A'
# optionally, can add spaces between pdb_aa_wt, pdb_resnu, and pdb_aa_mut
kortemme_baker_df['pdb_mutation'] = (
    kortemme_baker_df['pdb_aa_wt'] +
    kortemme_baker_df['pdb_resnum'].astype(str) +
    kortemme_baker_df['pdb_aa_mut']
)
kortemme_baker_df['ddg_exp'] = kortemme_baker_df['DDG_obs'].str.replace('>', '').astype(float)

In [10]:
display(kortemme_baker_df.head())
print(kortemme_baker_df.shape[0])

,pdb_id,Mutated_partner,pdb_chain,pdb_resnum,pdb_aa_wt,DDG_calc,DDG_obs,"Interface(O:no,1:yes)",PDBPosID,PubMedID,Location,FixedPDB_ID,FixedPDBResidueID,partner_pdb_chain,pdb_aa_mut,pdb_mutation,ddg_exp
0,1a22,hGH,A,14,M,0.00,0.1,0,1A2214,7504735,Table_2,NaN,NaN,NaN,A,M14A,0.1
1,1a22,hGH,A,18,H,1.84,-0.5,1,1A2218,7504735,Table_2,NaN,NaN,NaN,A,H18A,-0.5
2,1a22,hGH,A,21,H,0.78,0.2,1,1A2221,7504735,Table_2,NaN,NaN,NaN,A,H21A,0.2
3,1a22,hGH,A,22,Q,0.07,-0.2,1,1A2222,7504735,Table_2,NaN,NaN,NaN,A,Q22A,-0.2
4,1a22,hGH,A,25,F,1.30,-0.4,1,1A2225,7504735,Table_2,NaN,NaN,NaN,A,F25A,-0.4


380


In [11]:
kortemme_baker_df_1_bak = kortemme_baker_df.copy()

# Add SIFTS

In [12]:
kortemme_baker_df = kortemme_baker_df_1_bak.copy()

In [13]:
pdb_ids = sorted(set(kortemme_baker_df['pdb_id'].str.lower()))

In [14]:
sifts_dfs = {
    pdb_id: ascommon.pdb_tools.sifts.get_sifts_data(pdb_id)
    for pdb_id in pdb_ids
}

In [15]:
sifts_dfs['1a22'].head()

,comments,is_observed,pdb_aa,pdb_chain,pdb_id,pfam_id,resnum,uniprot_aa,uniprot_id,uniprot_position,residx
0,"T,loop",True,F,A,1a22,PF00103,1,F,P01241,27,1
1,"T,loop",True,P,A,1a22,PF00103,2,P,P01241,28,2
2,"T,loop",True,T,A,1a22,PF00103,3,T,P01241,29,3
3,"T,loop",True,I,A,1a22,PF00103,4,I,P01241,30,4
4,"T,loop",True,P,A,1a22,PF00103,5,P,P01241,31,5


In [16]:
# Add info to core
def get_sifts_data(pdb_id, pdb_chain, pdb_mutation):
    sifts_df = sifts_dfs[pdb_id]
    try:
        result = kmtools.pdb_tools.sifts.convert_pdb_mutations_to_uniprot(
            pdb_id, pdb_chain, pdb_mutation, sifts_df)
    except kmtools.pdb_tools.sifts.SIFTSError as e:
        logger.error(e)
        result = {}
    return result

results = [get_sifts_data(*x) for x in kortemme_baker_df[['pdb_id', 'pdb_chain', 'pdb_mutation']].values]

ERROR:root:No mutation mapping available! (1dn2, E, V10A, E, V10A): [{'uniprot_id': nan, 'pdb_chain': 'E', 'uniprot_pos': nan, 'uniprot_aa': nan, 'pfam_id': nan}]
ERROR:root:No mutation mapping available! (1dn2, E, W11A, E, W11A): [{'uniprot_id': nan, 'pdb_chain': 'E', 'uniprot_pos': nan, 'uniprot_aa': nan, 'pfam_id': nan}]
ERROR:root:No mutation mapping available! (1jrh, L, E27A, L, E27A): [{'uniprot_id': nan, 'pdb_chain': 'L', 'uniprot_pos': nan, 'uniprot_aa': nan, 'pfam_id': nan}]
ERROR:root:No mutation mapping available! (1jrh, L, D28A, L, D28A): [{'uniprot_id': nan, 'pdb_chain': 'L', 'uniprot_pos': nan, 'uniprot_aa': nan, 'pfam_id': nan}]
ERROR:root:No mutation mapping available! (1jrh, L, Y30A, L, Y30A): [{'uniprot_id': nan, 'pdb_chain': 'L', 'uniprot_pos': nan, 'uniprot_aa': nan, 'pfam_id': nan}]
ERROR:root:No mutation mapping available! (1jrh, L, Y91A, L, Y91A): [{'uniprot_id': nan, 'pdb_chain': 'L', 'uniprot_pos': nan, 'uniprot_aa': nan, 'pfam_id': nan}]
ERROR:root:No mutation

In [17]:
df = pd.DataFrame(results)
df.head()

,pfam_id_sifts,uniprot_id_sifts,uniprot_mutations_sifts
0,PF00103,P01241,M40A
1,PF00103,P01241,H44A
2,PF00103,P01241,H47A
3,PF00103,P01241,Q48A
4,PF00103,P01241,F51A


In [18]:
kortemme_baker_df['uniprot_id'] = df['uniprot_id_sifts']
kortemme_baker_df['uniprot_mutation'] = df['uniprot_mutations_sifts']
kortemme_baker_df['pfam_id'] = df['pfam_id_sifts']

In [19]:
pfam_a_clans = (
    pd.read_sql_table('pfam_a_clans', db_remote.engine, schema='pfam')
)

In [20]:
pfam_a_clans.head()

,pfam_id,clan_id,clan_name,pfam_name,pfam_description
0,PF00389,CL0325,Form_Glyc_dh,2-Hacid_dh,"D-isomer specific 2-hydroxyacid dehydrogenase,..."
1,PF00198,CL0149,CoA-acyltrans,2-oxoacid_dh,2-oxoacid dehydrogenases acyltransferase (cata...
2,PF04029,None,None,2-ph_phosp,2-phosphosulpholactate phosphatase
3,PF03171,CL0029,Cupin,2OG-FeII_Oxy,2OG-Fe(II) oxygenase superfamily
4,PF01073,CL0063,NADP_Rossmann,3Beta_HSD,3-beta hydroxysteroid dehydrogenase/isomerase ...


In [21]:
kortemme_baker_df['pfam_clan'] = kortemme_baker_df['pfam_id'].map(pfam_a_clans.set_index('pfam_id')['clan_id'])

In [22]:
kortemme_baker_df.tail()

,pdb_id,Mutated_partner,pdb_chain,pdb_resnum,pdb_aa_wt,DDG_calc,DDG_obs,"Interface(O:no,1:yes)",PDBPosID,PubMedID,Location,FixedPDB_ID,FixedPDBResidueID,partner_pdb_chain,pdb_aa_mut,pdb_mutation,ddg_exp,uniprot_id,uniprot_mutation,pfam_id,pfam_clan
375,3hfm,HEL,Y,96,K,2.13,7,1,3HFM525,10338006,Figure_1/Appendix,NaN,NaN,NaN,A,K96A,7.00,P00698,K114A,PF00062,CL0037
376,3hfm,HEL,Y,97,K,1.42,6.2,1,3HFM526,10338006,Figure_1/Appendix,NaN,NaN,NaN,A,K97A,6.20,P00698,K115A,PF00062,CL0037
377,3hfm,HEL,Y,98,I,0.10,-0.1,0,3HFM527,10338006,Figure_1,NaN,NaN,NaN,A,I98A,-0.10,P00698,I116A,PF00062,CL0037
378,3hfm,HEL,Y,100,S,0.95,0.25,1,3HFM529,10338006,Figure_1,NaN,NaN,NaN,A,S100A,0.25,P00698,S118A,PF00062,CL0037
379,3hfm,HEL,Y,101,D,1.02,1.5,1,3HFM530,10338006,Figure_1,NaN,NaN,NaN,A,D101A,1.50,P00698,D119A,PF00062,CL0037


In [23]:
assert kortemme_baker_df[kortemme_baker_df['FixedPDB_ID'].notnull()].empty

In [24]:
assert kortemme_baker_df[kortemme_baker_df['FixedPDBResidueID'].notnull()].empty

In [25]:
assert kortemme_baker_df['pfam_clan'].notnull().sum() > 100

In [26]:
kortemme_baker_df_2_bak = kortemme_baker_df.copy()

# Add Partner

In [27]:
kortemme_baker_df['partner_uniprot_id'] = np.nan

# Save to database

In [28]:
_header_columns = [
    'pdb_id', 'pdb_chain', 'partner_pdb_chain', 'pdb_mutation', 
    'uniprot_id', 'partner_uniprot_id', 'uniprot_mutation',
    'pfam_id', 'pfam_clan',
    'ddg_exp'
]

In [29]:
t = db.import_df(
    kortemme_baker_df[_header_columns],
    'kortemme_baker',
)

In [30]:
t.name

'kortemme_baker'

In [31]:
db.create_indexes(
    t.name,
    [(['pdb_id', 'pdb_chain', 'pdb_mutation'], True),
     (['uniprot_id', 'partner_uniprot_id', 'uniprot_mutation'], True)]
)

In [32]:
db.add_idx_column(t.name)

380